In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from datasets import load_dataset
import bitsandbytes
from huggingface_hub import notebook_login
import torch
import numpy as np

In [2]:
notebook_login()

comma = load_dataset("disi-unibo-nlp/COMMA", "it")

anto_ds = comma['train'].filter(lambda example : "abrogat" in example["full_text"].lower() or "antinom" in example["full_text"].lower())
anto_ds

Dataset({
    features: ['id', 'ruling_type', 'epigraph', 'body', 'decision', 'maxims_text', 'maxims_title', 'full_text', 'num_maxims', 'maxims_len', 'full_text_len', 'judgment_type', 'constitutional_parameters', 'maxims'],
    num_rows: 1860
})

In [5]:
models = {
    #"Saul": {'model_name': 'Equall/Saul-7B-Instruct-v1', 'context_window': 1024, 'prompt_function': lambda system_prompt, user_prompt: f"<|system|>\n{system_prompt}\n|<user>|\n{user_prompt}\n|<assistant>|\n\n"}, #Modello addestrato su testi legali
    #"Llamantino": {'model_name': 'swap-uniba/LLaMAntino-2-7b-hf-dolly-ITA', 'context_window': 8000, 'prompt_function': lambda system_prompt, user_prompt: f"Di seguito è riportata un'istruzione che descrive un'attività, abbinata ad un input che fornisce ulteriore informazione.\nScrivi una risposta che soddisfi adeguatamente la richiesta.\n\n### Istruzione:\n{system_prompt}\n\n### Input:\n{user_prompt}\n\n### Risposta:\n"}, # Doesn't work with transformers
    "Meta-Llama": {'model_name': 'meta-llama/Meta-Llama-3-8B-Instruct', 'context_window': 8000, 'prompt_function': lambda system_prompt, user_prompt: f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>{user_prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"},
    #"Falcon-7B": {'model_name': 'tiiuae/falcon-7b-instruct', 'context_window': 512, 'prompt_function': lambda system_prompt, user_prompt: f"User: {user_prompt}\nAssistant:{system_prompt}"},
    "Mixtral-8x22B": {'model_name': 'mistralai/Mixtral-8x22B-Instruct-v0.1', 'context_window': 1024, 'prompt_function': lambda system_prompt, user_prompt: f"[INST] {system_prompt} {user_prompt}\n[/INST]"},
    "Mixtral-7B": {'model_name': 'mistralai/Mistral-7B-Instruct-v0.2', 'context_window': 32000, 'prompt_function': lambda system_prompt, user_prompt: f"[INST] {system_prompt} {user_prompt}\n[/INST]"},
    #"Minerva-3B": {'model_name': 'sapienzanlp/Minerva-3B-base-v1.0', 'context_window': 512, 'prompt_function': lambda system_prompt, user_prompt: f"{system_prompt} {user_prompt}"}, # Modello italiano della Sapienza
    #"deepset/roberta-base-squad2" : {'model_name': 'deepset/roberta-base-squad2', 'context_window': 512}, # Modello per il question answering
    "Phi-small" : {'model_name': 'microsoft/Phi-3-small-4k-instruct', 'context_window': 8000, 'prompt_function': lambda system_prompt, user_prompt: f"<|system|>\n{system_prompt}\n|<user>|\n{user_prompt}\n|<assistant>|\n\n"},
    "Phi-medium" : {'model_name': 'microsoft/Phi-3-medium-4k-instruct', 'context_window': 8000, 'prompt_function': lambda system_prompt, user_prompt: f"<|system|>\n{system_prompt}\n|<user>|\n{user_prompt}\n|<assistant>|\n\n"},
    "Phi-medium-quantized" : {'model_name': 'kaitchup/Phi-3-medium-128k-instruct-awq-4bit', 'context_window': 8000, 'prompt_function': lambda system_prompt, user_prompt: f"<|system|>\n{system_prompt}\n|<user>|\n{user_prompt}\n|<assistant>|\n\n"},
}

def init_model(model_name, quantization_config=0):
    if quantization_config == 0:
        model = AutoModelForCausalLM.from_pretrained(model_name, device_map="cuda", torch_dtype=torch.bfloat16)
    elif quantization_config == 1:
        bnb_config = BitsAndBytesConfig(
                                    load_in_4bit=True,
                                    bnb_4bit_use_double_quant=True,
                                    bnb_4bit_quant_type="nf4",
                                    bnb_4bit_compute_dtype=torch.bfloat16,
                                )
        model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map="cuda")
        
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    return model, tokenizer

In [6]:
# load the model
model = None
model, tokenizer = init_model(models["Mixtral-7B"]["model_name"], 0)

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [11]:
valid_texts = np.array([])

for item in anto_ds["full_text"]:
    if any(word in item for word in ["antinomi", "antonim"]) and len(item) < 31000:
        valid_texts = np.append(valid_texts, item)
        
valid_texts.shape

(62,)

In [12]:
results = []

for i, text in enumerate(valid_texts):
    if i > 3:
        break
    inputs = tokenizer.encode(models["Meta-Llama"]["prompt_function"]("You are a legal expert. I am going to give you a text, extract all pairs of antinomian laws. Answer directly without any preamble or comment. Answer always using a correct Italian. Answer in few sentences.", text),
                                return_tensors="pt",
                                truncation=True).to("cuda")
    outputs = model.generate(inputs, max_length=31000, num_return_sequences=1)
    output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    results.append([text, output])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


ValueError: Input length of input_ids is 7906, but `max_length` is set to 7500. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.

In [ ]:
for result in results:
    print(f"Input: {result[0]}\n\nOutput: {result[1]}\n\n")

Input: Nel giudizio di legittimità costituzionale degli artt. 19, comma 2, 22, 23 e 27, commi 1 e 3, della legge Regione Sicilia approvata il 28 luglio 1997, recante "Misure di politiche attive del lavoro in Sicilia. Modifiche alla legge regionale 21 dicembre 1995, n. 85. Norme in materia di Attività produttive e di Sanità. Disposizioni varie" promosso con ricorso del Commissario dello Stato per la Regione Siciliana, notificato il 5 agosto 1997, depositato in Cancelleria il 13 successivo ed iscritto al n. 52 del registro ricorsi 1997; Udito nell'udienza pubblica del 9 febbraio 1999 il giudice relatore Piero Alberto Capotosti. RITENUTO IN FATTO. 1. - Con ricorso notificato il 5 agosto 1997, e depositato il successivo 13 agosto, il Commissario dello Stato per la Regione Siciliana ha impugnato gli artt. 19, comma 2, 22, 23 e 27, commi 1 e 3, della legge approvata dall'Assemblea regionale siciliana il 28 luglio 1997, recante "Misure di politiche attive del lavoro in Sicilia. Modifiche alla